In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cPickle

In [2]:
tags_table = pd.read_excel('../data/osago_tags_hierarchy.xlsx', header=None, 
                           names=['tag', 'parent', 'support', 'answer'])
tags_table.head()

,tag,parent,support,answer
0,Документы в СК,ОСАГО,документы в страховую компанию по осаго; какие...,"Так, вы хотели бы утонить список документов дл..."
1,причинении вреда имуществу потерпевшего,Документы в СК,Список документов по ОСАГО при причинении вред...,NaN
2,причинен ущерб здоровью,Документы в СК,возмещение ущерба здоровью осаго; Осаго здоров...,NaN
3,утрата профессиональной трудоспособности,Документы в СК,утрата трудоспособности при дтп ; потеря трудо...,NaN
4,причинения вреда жизни потерпевшего,Документы в СК,осаго смерть потерпевшего; выплаты осаго в слу...,NaN


In [3]:
# Leaf indicator
tags_table['leaf'] = tags_table.tag.apply(lambda t: t not in tags_table.parent.unique())
# Adding answers
tags_table.loc[np.logical_and(tags_table.leaf == False, tags_table.answer.apply(lambda a: a is np.nan)), 
               'answer'] = u'Уточните ваш вопрос'

In [4]:
tag_names = tags_table.tag.values

---

In [5]:
import pymorphy2
from nltk.tokenize import RegexpTokenizer

morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer('\w+')

def normalize(s):
    return ' '.join(map(lambda w: morph.parse(w)[0].normal_form, tokenizer.tokenize(s)))

def get_normalized_tokens(s):
    return map(lambda w: morph.parse(w)[0].normal_form, tokenizer.tokenize(s))

def remove_minor_pos(s):
    return ' '.join(filter(lambda w: morph.parse(w)[0].tag.POS not in ['PREP', 'CONJ', 'INTJ'], tokenizer.tokenize(s)))

def normalize_remove(s):
    return normalize(remove_minor_pos(s))

In [37]:
def count_part_entries2(words, text, part_size, pair_weight):
    count = 0
    words = words.split(' ')
    words = map(lambda w: w[:part_size], words)
    text = ' '.join(map(lambda w: w[:part_size], text.split(' ')))
    for i in range(len(words)-1):
        if words[i] in text:
            count += 1
            pair = ' '.join((words[i], words[i+1]))
            if pair in text:
                count += pair_weight
    if words[-1] in text:
        count += 1
    return count

In [38]:
def get_top_tags(text_normed, tags, num_tags, part_size, pair_weight=1):
    similarities = []
    for tag in tags:
        support = tags_table[tags_table.norm == tag].support.values[0]
        variants = [tag]
        if support is not np.nan:
            variants += support.split(';')
        scores = map(lambda var:\
            count_part_entries2(var, text_normed, part_size, pair_weight) / (len(var.split(' ')) + 0.1), variants)
        similarities.append(np.max(scores))
    result = []
    for _ in range(num_tags):
        idx = np.argmax(similarities)
        result.append((tags[idx], similarities[idx]))
        similarities[idx] = 0.
    return result

In [39]:
tags_table.loc[:, 'support'] = tags_table.support.apply(\
        lambda sup: ';'.join(map(normalize_remove, sup.split(';'))) if sup is not np.nan else sup)

In [40]:
tags_normed = map(normalize_remove, tag_names)

In [41]:
tags_table['norm'] = tags_normed

In [42]:
cPickle.dump(tags_table, open('../data/tags_table.p', 'wb'))

In [43]:
tags_table.ix[list(tags_table[tags_table.parent == u'ОСАГО'].index)]

,tag,parent,support,answer,leaf,norm
0,Документы в СК,ОСАГО,документ страховой компания осаго;какой докуме...,"Так, вы хотели бы утонить список документов дл...",False,документ ск
5,выплаты по ОСАГО,ОСАГО,выплата осаго дтп;выплата осаго;возмещение вре...,"Так, у вас возник вопрос связанный с выплатами...",False,выплата осаго
15,договор страхования ОСАГО,ОСАГО,страховой договор осаго;страховой полис осаго,Уточните ваш вопрос,False,договор страхование осаго
18,ответственность по ОСАГО,ОСАГО,штраф осаго;ответственность осаго,Уточните ваш вопрос,False,ответственность осаго
24,ремонт по ОСАГО,ОСАГО,ремонт автомобиль осаго;стоимость ремонт осаго...,Уточните ваш вопрос,False,ремонт осаго
27,Полис,ОСАГО,полис осаго,"Так, у вас возникли проблемы с полисом ОСАГО. ...",False,полис


In [44]:
sessions = dict()

def predict_tag_for_request(tag_id, question_body, sender_id, session_new):
    cur_tag = u'ОСАГО'
    history = ''
    if sender_id in sessions:
        if not session_new:
            cur_tag, history = sessions[sender_id]
        else:
            sessions[sender_id] = None

    if tags_table[tags_table.tag == cur_tag].leaf.values[0]:
        return tags_table[tags_table.tag == cur_tag].index.values[0], 404

    tag_indices = []
    parents = [cur_tag]
    while parents:
        p = parents.pop()
        tag_indices += list(tags_table[tags_table.parent == p].index)
        parents += list(tags_table[tags_table.parent == p].tag)
    local_table = tags_table.ix[tag_indices]
    tags_norm = local_table.norm.values

    question_normed = normalize(remove_minor_pos(question_body))
    history += ' ' + question_normed
    top_tag_normed, similarity = get_top_tags(history, tags_norm, 1, 5)[0]
    top_tag = local_table[tags_table.norm == top_tag_normed].tag.values[0]

    sessions[sender_id] = (top_tag, history)

    return tags_table[tags_table.tag == top_tag].index.values[0], similarity

In [45]:
idx, sim = predict_tag_for_request(0, u'Страховая не платит по ОСАГО, хочу судиться', 3, True)
print tags_table.ix[idx].tag
ans = tags_table.ix[idx].answer
if ans is not np.nan:
    print ans

Отказ в выплате


/home/ilya/.virtualenvs/lawyer/lib/python2.7/site-packages/ipykernel/__main__.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [46]:
# var = normalize_remove(u'максимальный размер выплаты по осаго')
var = normalize_remove(u'сколько платит страховая компания по осаго')
# var = normalize_remove(u'страховая компания не платит')
print var
text = normalize_remove(u'Страховая не платит по ОСАГО, хочу судиться')
print text
count_part_entries2(var, text, 5, 2) / (len(var.split(' ')) + 0.1)

платить страховой компания осаго
страховой не платить осаго хотеть судиться


0.7317073170731708

In [47]:
idx, sim = predict_tag_for_request(0, u'ОСАГО', 3, True)
print tags_table.ix[idx].tag
ans = tags_table.ix[idx].answer
if ans is not np.nan:
    print ans

выплаты по ОСАГО
Так, у вас возник вопрос связанный с выплатами по ОСАГО. Опишите, пожалуйста, подробней в чем именно у Вас проблема с выплатами по ОСАГО?.


/home/ilya/.virtualenvs/lawyer/lib/python2.7/site-packages/ipykernel/__main__.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [17]:
var = normalize_remove(u'ОСАГО')
var = normalize_remove(u'выплаты по осаго')
print var
text = normalize_remove(u'ОСАГО')
print text
count_part_entries2(var, text, 5, 2) / (len(var.split(' ')) + 0.1)

выплата осаго
осаго


0.47619047619047616

In [54]:
var = normalize_remove(u'регресс по ОСАГО')
var = normalize_remove(u'договор страхования ОСАГО')
print var
text = normalize_remove(u'Регрессивный иск от страховой по ОСАГО')
print text
count_part_entries2(var, text, 5, 0.5) / (len(var.split(' ')) + 0.1)

договор страхование осаго
регрессивный иск страховой осаго


0.8064516129032258

In [26]:
var = normalize_remove(u'Виновник ДТП без полиса ОСАГО')

print var
text = normalize_remove(u'Взыскание с виновника дтп без учета взноса')
print text
count_part_entries2(var, text, 5, 2) / (len(var.split(' ')) + 0.1)

отсутствие полис осаго виновник дтп
взыскание виновник дтп учёт взнос


0.7843137254901962